In [1]:
%pip install pandas pinecone python-dotenv openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/745.9 kB ? eta -:--:--
   --------------------------------------- 745.9/745.9 kB 15.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 50.7 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
Using cached pydantic-2.12.4-py3-none-any.whl (463 kB)
   -----------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\deepa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

True

In [3]:
token= os.getenv("RUNPOD_TOKEN") 
open_ai_base_url = os.getenv("RUNPOD_EMBEDDING_URL") 
model_name= os.getenv("MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [4]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

## Try out embeddings

In [5]:
output = client.embeddings.create(input = ["helloo there"],model=model_name)
embedings = output.data[0].embedding
print(embedings)

[-0.054519448429346085, -0.057440135627985, 0.08616019785404205, -0.06376828253269196, 0.017524108290672302, -0.012412910349667072, 0.05159876495599747, 0.052328936755657196, 0.02945023775100708, -0.022635307163000107, -0.013690710067749023, -0.04770451784133911, 0.029328543692827225, 0.03212753310799599, 0.05646657198667526, -0.009005445055663586, 0.012899691238999367, -0.054032668471336365, -0.1022239699959755, -0.02020140364766121, 0.030058713629841805, 0.05159876495599747, -0.0287200678139925, -0.035778388381004333, 0.0014375245664268732, -0.007575526367872953, 0.01947123184800148, 0.035048216581344604, -0.008214426226913929, -0.07350390404462814, -0.0010496211471036077, -0.012656300328671932, 0.04648756608366966, -0.0009963794145733118, 0.05159876495599747, 0.0001302518940065056, 0.05865708738565445, -0.025312600657343864, -0.07788492739200592, -0.005567555315792561, 0.06011742725968361, -0.024825820699334145, -0.0020840303041040897, -0.016307156533002853, 0.02300039306282997, -0.

In [6]:
len(embedings)

384

## Wrangle dataset

In [7]:
df=pd.read_json('products/products.jsonl',lines=True)

In [8]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [9]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [10]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [11]:
texts = df['text'].tolist()

In [12]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [14]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

## Generate Embeddings

In [15]:
output = client.embeddings.create(input = texts,model=model_name)

In [16]:
embeddings = output.data

## Push data to database

In [17]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "coffeeshop",
    "metric": "cosine",
    "host": "coffeeshop-c1kyiyw.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null,
    "_response_info": {
        "raw_headers": {
            "content-type": "application/json",
            "access-control-allow-origin": "*",
            "vary": "origin,access-control-request-method,access-control-request-headers",
            "access-control-expose-headers": "*",
            "x-pinecone-api-version": "2025-10"

In [18]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

C:\Users\deepa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


UpsertResponse(upserted_count=21, _response_info={'raw_headers': {'date': 'Mon, 24 Nov 2025 23:24:45 GMT', 'content-type': 'application/json', 'content-length': '20', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '42140', 'x-pinecone-request-latency-ms': '1217', 'x-pinecone-request-id': '601144023074409280', 'x-envoy-upstream-service-time': '930', 'grpc-status': '0', 'server': 'envoy'}})

## Get Closest documents

In [19]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [20]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

QueryResponse(matches=[{'id': 'Cappuccino',
 'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino made with '
                      'freshly brewed espresso, steamed milk, and a frothy '
                      'milk cap. This delightful drink offers a perfect '
                      'balance of bold coffee flavor and smooth milk, making '
                      'it an ideal companion for relaxing mornings or lively '
                      "conversations. -- Ingredients: ['Espresso', 'Steamed "
                      "Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
 'score': 0.733623564,
 'values': []}, {'id': 'Sugar Free Vanilla syrup',
 'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet flavor of '
                      'vanilla without the sugar, making it perfect for your '
                      'coffee or dessert. This syrup offers a guilt-free way '
                      'to enhance your beverages, ensuring you never miss out '
                      "on f